# Script for using linear mixed effects model to quantify differences in synaptic proteins in 5xFAD vs. WT

In [64]:
#most of the code is taken/adapted from this tutorial: https://towardsdatascience.com/how-to-run-linear-mixed-effects-models-in-python-jupyter-notebooks-4f8079c4b589

In [65]:
# Load packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os

In [66]:
parentdir = '/Users/margaret/Dropbox (MIT)/mExR_analysis/SmallData/2023.05_5xFAD/'

In [81]:
os.listdir(parentdir)

['mExR_5xFAD_2023.05_reg-error.pzfx',
 '5xFAD_2023.05_synapses_WT_vs_5xFAD_RE.pzfx',
 '5xFAD_vs_WT_n=3_Abeta.xlsx',
 '5xFAD_2023.05_abeta_WT_vs_5xFAD_RE.pzfx',
 '5xFAD_2023.05_abeta_analysis_updated.pzfx',
 '5xFAD_vs_WT_n=2_Abeta.csv',
 'mExR_2023.05_5xFAD.xlsx',
 '5xFAD_vs_WT_n=3_Abeta.csv',
 '5xFAD_vs_WT_n=2_synapses.csv',
 '5xFAD_vs_WT_n=3_synapses.csv',
 '5xFAD_2022.05_camkiia.pzfx']

In [82]:
data = pd.read_csv(parentdir + '5xFAD_vs_WT_n=2_synapses.csv')

In [83]:
data

,Animal,ROI,Group,RIM1_vol,GluA3_vol,NR2B_vol,RIMBP_vol,GluA2_vol,GluA1_vol,NR1_vol,Shank3_vol,Homer1_vol,GluA4_vol,PSD95_vol,Bassoon_vol,SynGAP_vol,IRsp53_vol
0,5xFAD-2,ROI1,5xFAD,1.275974,0.314171,0.353728,0.752473,0.035785,0.426865,0.253600,0.432301,1.050779,0.290645,0.421013,0.657133,0.391745,0.738020
1,5xFAD-2,ROI2,5xFAD,0.893086,0.276639,0.356786,0.546142,0.048083,0.292006,0.329737,0.522841,0.758914,0.342606,0.415196,0.646244,0.389374,0.611348
2,5xFAD-2,ROI3,5xFAD,0.794513,0.229565,0.321661,0.379173,0.053106,0.203572,0.277443,0.483429,0.554174,0.175842,0.363866,0.548275,0.320136,0.415166
3,5xFAD-2,ROI4,5xFAD,1.119313,0.345086,0.435711,0.725434,0.042758,0.359016,0.401425,0.528090,1.056154,0.352155,0.478293,0.437506,0.435614,0.390674
4,5xFAD-2,ROI5,5xFAD,0.959586,0.333744,0.439819,0.647004,0.045960,0.378807,0.421723,0.536081,0.767107,0.486110,0.411397,0.648290,0.421976,0.619811
5,5xFAD-3,ROI1,5xFAD,0.734355,0.412959,0.483517,0.728528,0.093835,0.332874,0.455124,0.547621,0.995113,0.332731,0.578061,0.803573,0.615044,0.776124
6,5xFAD-3,ROI2,5xFAD,0.851018,0.298961,0.387346,0.583731,0.035236,0.285182,0.378741,0.492032,0.786435,0.187197,0.489924,0.610938,0.476547,0.583583
7,5xFAD-3,ROI3,5xFAD,1.025678,0.294000,0.347359,0.455604,0.006476,0.272515,0.351633,0.493225,0.740646,0.116091,0.452495,0.589812,0.382915,0.297917
8,5xFAD-3,ROI4,5xFAD,0.675139,0.407433,0.362534,0.551104,0.012189,0.252572,0.333725,0.404180,0.590964,0.237453,0.352896,0.540447,0.354063,0.444745
9,WT-2,ROI1,WT,1.177196,0.488599,0.709457,1.117161,0.003533,0.539853,0.597186,0.853723,1.122522,0.224530,0.871557,1.040250,0.856264,0.857650


In [70]:
# Run mixed lm for RIM1 volume
md = smf.mixedlm("RIM1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: RIM1_vol
No. Observations: 17      Method:             REML    
No. Groups:       4       Scale:              0.0664  
Min. group size:  4       Log-Likelihood:     -4.9110 
Max. group size:  5       Converged:          Yes     
Mean group size:  4.2                                 
------------------------------------------------------
              Coef. Std.Err.   z   P>|z| [0.025 0.975]
------------------------------------------------------
Intercept     0.917    0.220 4.166 0.000  0.485  1.348
Group[T.WT]   0.408    0.313 1.305 0.192 -0.205  1.020
Group Var     0.082    0.410                          



In [71]:
# Run mixed lm for GluA3 volume
md = smf.mixedlm("GluA3_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA3_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0039   
Min. group size:  4       Log-Likelihood:     18.0987  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.324    0.021 15.219 0.000  0.282  0.365
Group[T.WT]   0.149    0.031  4.849 0.000  0.089  0.209
Group Var     0.000    0.018                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [72]:
for x in range (0, 2):
    print(mdf.pvalues[x])

2.632301581255709e-52
1.242624941048332e-06


In [74]:
# Run mixed lm for NR2B volume
md = smf.mixedlm("NR2B_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NR2B_vol
No. Observations: 17      Method:             REML    
No. Groups:       4       Scale:              0.0039  
Min. group size:  4       Log-Likelihood:     18.2166 
Max. group size:  5       Converged:          Yes     
Mean group size:  4.2                                 
------------------------------------------------------
             Coef. Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------
Intercept    0.388    0.021 18.648 0.000  0.347  0.428
Group[T.WT]  0.206    0.030  6.807 0.000  0.147  0.266
Group Var    0.000    0.017                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [75]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.3021674334620696e-77
9.931907277466528e-12


In [84]:
# Run mixed lm for RIM-BP volume
md = smf.mixedlm("RIMBP_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: RIMBP_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0129   
Min. group size:  4       Log-Likelihood:     7.6906   
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
               Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept      0.595    0.083 7.155 0.000  0.432  0.758
Group[T.WT]    0.336    0.118 2.838 0.005  0.104  0.568
Group Var      0.011    0.134                          



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)


In [85]:
for x in range (0, 2):
    print(mdf.pvalues[x])

8.365276050964072e-13
0.004535372051967385


In [86]:
# Run mixed lm for GluA2 volume
md = smf.mixedlm("GluA2_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA2_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0003   
Min. group size:  4       Log-Likelihood:     36.4075  
Max. group size:  5       Converged:          No       
Mean group size:  4.2                                  
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.041    0.006  6.759 0.000  0.029  0.054
Group[T.WT]  -0.037    0.009 -4.084 0.000 -0.054 -0.019
Group Var     0.000                                    



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 0.039720
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_l

In [87]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.3848813520306577e-11
4.421630076884965e-05


In [88]:
# Run mixed lm for GluA1 volume
md = smf.mixedlm("GluA1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA1_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0038   
Min. group size:  4       Log-Likelihood:     18.2858  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
              Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.311    0.022 14.250 0.000  0.268  0.354
Group[T.WT]   0.171    0.032  5.397 0.000  0.109  0.233
Group Var     0.000    0.017                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [89]:
for x in range (0, 2):
    print(mdf.pvalues[x])

4.483235853500648e-46
6.779263820307266e-08


In [91]:
# Run mixed lm for NR1 volume
md = smf.mixedlm("NR1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

        Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: NR1_vol
No. Observations: 17      Method:             REML   
No. Groups:       4       Scale:              0.0043 
Min. group size:  4       Log-Likelihood:     17.4321
Max. group size:  5       Converged:          Yes    
Mean group size:  4.2                                
-----------------------------------------------------
            Coef. Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------
Intercept   0.356    0.023 15.708 0.000  0.311  0.400
Group[T.WT] 0.170    0.032  5.247 0.000  0.107  0.234
Group Var   0.000    0.036                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [92]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.3348937930633131e-55
1.5452036622680153e-07


In [93]:
# Run mixed lm for Shank3 volume
md = smf.mixedlm("Shank3_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Shank3_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0037    
Min. group size:  4       Log-Likelihood:     17.3839   
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.493    0.038 12.977 0.000  0.418  0.567
Group[T.WT]    0.297    0.054  5.480 0.000  0.191  0.403
Group Var      0.002    0.053                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [94]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.6576822258150464e-38
4.2620434179632984e-08


In [95]:
# Run mixed lm for Homer1 volume
md = smf.mixedlm("Homer1_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Homer1_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0231    
Min. group size:  4       Log-Likelihood:     4.8346    
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.811    0.051 15.878 0.000  0.711  0.911
Group[T.WT]    0.170    0.074  2.294 0.022  0.025  0.315
Group Var      0.000    0.068                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [96]:
for x in range (0, 2):
    print(mdf.pvalues[x])

8.975889214991036e-57
0.021800842715938577


In [97]:
# Run mixed lm for GluA4 volume
md = smf.mixedlm("GluA4_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: GluA4_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0065   
Min. group size:  4       Log-Likelihood:     13.6162  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept     0.277    0.040  6.900 0.000  0.198  0.355
Group[T.WT]  -0.122    0.057 -2.132 0.033 -0.235 -0.010
Group Var     0.002    0.043                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [98]:
for x in range (0, 2):
    print(mdf.pvalues[x])

5.203880528305055e-12
0.03298431277497725


In [99]:
# Run mixed lm for PSD95 volume
md = smf.mixedlm("PSD95_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: PSD95_vol
No. Observations: 17      Method:             REML     
No. Groups:       4       Scale:              0.0039   
Min. group size:  4       Log-Likelihood:     16.3262  
Max. group size:  5       Converged:          Yes      
Mean group size:  4.2                                  
-------------------------------------------------------
               Coef. Std.Err.   z   P>|z| [0.025 0.975]
-------------------------------------------------------
Intercept      0.443    0.054 8.213 0.000  0.337  0.548
Group[T.WT]    0.379    0.077 4.956 0.000  0.229  0.529
Group Var      0.005    0.102                          



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [100]:
for x in range (0, 2):
    print(mdf.pvalues[x])

2.1515191723146315e-16
7.182068179035072e-07


In [101]:
# Run mixed lm for Bassoon volume
md = smf.mixedlm("Bassoon_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: Bassoon_vol
No. Observations: 17      Method:             REML       
No. Groups:       4       Scale:              0.0114     
Min. group size:  4       Log-Likelihood:     9.7584     
Max. group size:  5       Converged:          No         
Mean group size:  4.2                                    
---------------------------------------------------------
                Coef. Std.Err.   z    P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept       0.610    0.043 14.213 0.000  0.526  0.694
Group[T.WT]     0.413    0.062  6.670 0.000  0.292  0.534
Group Var       0.001    0.045                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2207: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2219: ConvergenceWarning: Gradient optimization failed, |grad| = 0.216830
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_l

In [102]:
for x in range (0, 2):
    print(mdf.pvalues[x])

7.641889945051892e-46
2.5552027001624342e-11


In [103]:
# Run mixed lm for SynGAP volume
md = smf.mixedlm("SynGAP_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: SynGAP_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0083    
Min. group size:  4       Log-Likelihood:     12.4312   
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.421    0.032 13.325 0.000  0.359  0.483
Group[T.WT]    0.391    0.046  8.529 0.000  0.301  0.480
Group Var      0.000    0.024                           



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [104]:
for x in range (0, 2):
    print(mdf.pvalues[x])

1.6674209810353732e-40
1.4710954940054725e-17


In [105]:
# Run mixed lm for IRsp53 volume
md = smf.mixedlm("IRsp53_vol ~ Group", data, groups=data["Animal"])
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

         Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: IRsp53_vol
No. Observations: 17      Method:             REML      
No. Groups:       4       Scale:              0.0186    
Min. group size:  4       Log-Likelihood:     6.4447    
Max. group size:  5       Converged:          Yes       
Mean group size:  4.2                                   
--------------------------------------------------------
               Coef. Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept      0.542    0.045 11.919 0.000  0.453  0.631
Group[T.WT]    0.211    0.066  3.183 0.001  0.081  0.341
Group Var      0.000                                    



/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:1635: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2238: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/margaret/miniconda3/envs/lmer/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2262: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


In [106]:
for x in range (0, 2):
    print(mdf.pvalues[x])

9.438356714657923e-33
0.0014570906176644852
